In [ ]:
%defaultDatasource jdbc:h2:mem:db

In [ ]:
DROP TABLE IF EXISTS Filme;
DROP TABLE IF EXISTS Avaliador;
DROP TABLE IF EXISTS Avaliacao;
DROP TABLE IF EXISTS Franquia;
DROP TABLE IF EXISTS FranquiaFilme;
DROP TABLE IF EXISTS Genero;
DROP TABLE IF EXISTS GeneroFilme;
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS PessoaFilme;
DROP TABLE IF EXISTS Sequencia;
DROP TABLE IF EXISTS Streaming;
DROP TABLE IF EXISTS StreamingFilme;

In [ ]:
CREATE TABLE Filme (
    id_TMDB INTEGER,
    id_IMDB VARCHAR(20),
    titulo VARCHAR(200),
    titulo_original VARCHAR(200),
    sinopse VARCHAR(1000),
    duracao INTEGER,
    ano INTEGER,
    classificacao VARCHAR(55),
    situacao VARCHAR(20),
    idioma_original VARCHAR(5),
    orcamento FLOAT,
    receita FLOAT,
    num_oscars FLOAT,
    PRIMARY KEY(id_TMDB)
) AS SELECT
    id_TMDB,
    id_IMDB,
    titulo,
    titulo_original,
    sinopse,
    duracao,
    ano,
    classificacao,
    situacao,
    idioma_original,
    orcamento,
    receita,
    num_oscars
FROM CSVREAD('../data/processed/Filme.csv');

In [ ]:
CREATE TABLE Avaliador (
    id VARCHAR(15),
    nome VARCHAR(15),
    PRIMARY KEY(id)
) AS SELECT
    id,
    nome
FROM CSVREAD('../data/processed/Avaliador.csv');

CREATE TABLE Avaliacao (
    id_avaliador VARCHAR(15),
    id_filme INTEGER,
    nota FLOAT,
    PRIMARY KEY(id_avaliador, id_filme),
    FOREIGN KEY(id_avaliador)
      REFERENCES Avaliador(id)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    id_avaliador,
    id_filme_TMDB,
    nota
FROM CSVREAD('../data/processed/Avaliacao.csv');

In [ ]:
CREATE TABLE Franquia (
    nome VARCHAR(200),
    PRIMARY KEY(nome)
) AS SELECT
    nome
FROM CSVREAD('../data/processed/Franquia.csv');

CREATE TABLE FranquiaFilme (
    nome_franquia VARCHAR(200),
    id_filme INTEGER,
    PRIMARY KEY(nome_franquia, id_filme),
    FOREIGN KEY(nome_franquia)
      REFERENCES Franquia(nome)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    nome_franquia,
    id_filme_TMDB
FROM CSVREAD('../data/processed/FranquiaFilme.csv');

CREATE TABLE Genero (
    nome VARCHAR(50),
    PRIMARY KEY(nome)
) AS SELECT
    nome
FROM CSVREAD('../data/processed/Genero.csv');

CREATE TABLE GeneroFilme (
    nome_genero VARCHAR(50),
    id_filme INTEGER,
    PRIMARY KEY(nome_genero, id_filme),
    FOREIGN KEY(nome_genero)
      REFERENCES Genero(nome)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    nome_genero,
    id_filme_TMDB
FROM CSVREAD('../data/processed/GeneroFilme.csv');

In [ ]:
CREATE TABLE Pessoa (
    id_TMDB INTEGER,
    id_IMDB VARCHAR(20),
    nome VARCHAR(200),
    nacionalidade VARCHAR(300),
    num_oscars INTEGER,
    PRIMARY KEY(id_TMDB)
) AS SELECT
    id_TMDB,
    id_IMDB,
    nome,
    nacionalidade,
    num_oscars
FROM CSVREAD('../data/processed/Pessoa.csv');

CREATE TABLE PessoaFilme (
    id_pessoa INTEGER,
    id_filme INTEGER,
    ator BOOL,
    diretor BOOL,
    roteirista BOOL,
    PRIMARY KEY(id_pessoa, id_filme),
    FOREIGN KEY(id_pessoa)
      REFERENCES Pessoa(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    id_pessoa_TMDB,
    id_filme_TMDB,
    ator,
    diretor,
    roteirista
FROM CSVREAD('../data/processed/PessoaFilme.csv');

In [ ]:
CREATE TABLE Sequencia (
    id_filme INTEGER,
    id_filme_sequencia INTEGER,
    PRIMARY KEY(id_filme, id_filme_sequencia),
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme_sequencia)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    id_filme_TMDB,
    id_filme_sequencia_TMDB
FROM CSVREAD('../data/processed/Sequencia.csv');

CREATE TABLE Streaming (
    nome VARCHAR(150),
    PRIMARY KEY(nome)
) AS SELECT
    nome
FROM CSVREAD('../data/processed/Streaming.csv');

CREATE TABLE StreamingFilme (
    nome_streaming VARCHAR(150),
    id_filme INTEGER,
    PRIMARY KEY(nome_streaming, id_filme),
    FOREIGN KEY(nome_streaming)
      REFERENCES Streaming(nome)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    nome_streaming,
    id_filme_TMDB
FROM CSVREAD('../data/processed/StreamingFilme.csv');

In [ ]:
SELECT * FROM Filme;
SELECT * FROM Avaliador;
SELECT * FROM Avaliacao;
SELECT * FROM Franquia;
SELECT * FROM FranquiaFilme;
SELECT * FROM Genero;
SELECT * FROM GeneroFilme;
SELECT * FROM Pessoa;
SELECT * FROM PessoaFilme;
SELECT * FROM Sequencia;
SELECT * FROM Streaming;
SELECT * FROM StreamingFilme;

# Queries no Dataset

In [ ]:
/* Relação entre sucesso com o público (receita)
   e sucesso com a crítica (nota média) dos filmes */

DROP TABLE IF EXISTS FilmeReceitaNota;
DROP TABLE IF EXISTS FilmeAvaliacao;

CREATE VIEW FilmeAvaliacao AS
    SELECT A.id_filme, SUM(A.nota) nota_total, COUNT(A.id_filme) qtd_avaliacoes
        FROM Avaliacao A
        GROUP BY A.id_filme;

CREATE VIEW FilmeReceitaNota AS
    SELECT A.id_filme, F.titulo, F.ano, F.receita, (A.nota_total / A.qtd_avaliacoes) nota_media
        FROM Filme F, FilmeAvaliacao A
        WHERE A.id_filme = F.id_TMDB
          AND qtd_avaliacoes > 2;

-- Ordenação decrescente por receita
SELECT titulo, receita, nota_media 
    FROM FilmeReceitaNota
    ORDER BY receita DESC LIMIT 10;

-- Ordenação decrescente por nota média
SELECT titulo, receita, nota_media
    FROM FilmeReceitaNota
    ORDER BY nota_media DESC LIMIT 10;

In [ ]:
/* Gênero que fez mais sucesso com a crítica, isto é, 
   que tem os filmes com as melhores notas médias, na década de 2000 */

DROP TABLE IF EXISTS GeneroCritica;

CREATE VIEW GeneroCritica AS
    SELECT G.nome, SUM(F.nota_media) nota_total, COUNT(F.id_filme) qtd_filme
        FROM Genero G, FilmeReceitaNota F, GeneroFilme GF
        WHERE G.nome = GF.nome_genero
              AND F.id_filme = GF.id_filme
              AND (F.ano >= 2000 AND F.ano < 2010)
        GROUP BY G.nome;

SELECT nome 
    FROM GeneroCritica
    WHERE (nota_total / qtd_filme) = (SELECT MAX(nota_total / qtd_filme)
                                          FROM GeneroCritica);

In [ ]:
/* Gênero que fez mais sucesso o público, isto é, 
   que tem os filmes com a maior renda média, na década de 2000 */

DROP TABLE IF EXISTS GeneroPublico;

CREATE VIEW GeneroPublico AS
    SELECT G.nome, SUM(F.receita) receita_total, COUNT(F.id_filme) qtd_filme
        FROM Genero G, FilmeReceitaNota F, GeneroFilme GF
        WHERE G.nome = GF.nome_genero
              AND F.id_filme = GF.id_filme
              AND ((F.ano >= 2000) AND (F.ano < 2010))
        GROUP BY G.nome;
        
SELECT nome
    FROM GeneroPublico
    WHERE (receita_total / qtd_filme) = (SELECT MAX(receita_total / qtd_filme)
                                             FROM GeneroPublico);

In [ ]:
/* Franquia que fez mais sucesso com a crítica,
   isto é, que tem os filmes com a maior nota média */

DROP TABLE IF EXISTS FranquiaCritica;

CREATE VIEW FranquiaCritica AS
    SELECT FR.nome, SUM(FI.nota_media) nota_total, COUNT(FF.id_filme) qtd_filme
        FROM Franquia FR, FilmeReceitaNota FI, FranquiaFilme FF 
        WHERE FR.nome = FF.nome_franquia
              AND FI.id_filme = FF.id_filme
        GROUP BY FR.nome;
        
SELECT nome
    FROM FranquiaCritica
    WHERE (nota_total / qtd_filme) = (SELECT MAX(nota_total / qtd_filme)
                                          FROM FranquiaCritica);

In [ ]:
/* Franquia que fez mais sucesso com o público,
   isto é, que tem os filmes com a maior renda média */

DROP TABLE IF EXISTS FranquiaPublico;

CREATE VIEW FranquiaPublico AS
    SELECT FR.nome, SUM(FI.receita) receita_total, COUNT(FF.id_filme) qtd_filme
        FROM Franquia FR, Filme FI, FranquiaFilme FF 
        WHERE FR.nome = FF.nome_franquia
              AND FI.id_TMDB = FF.id_filme
        GROUP BY FR.nome;
        
SELECT nome
    FROM FranquiaPublico
    WHERE (receita_total / qtd_filme) = (SELECT MAX(receita_total / qtd_filme)
                                             FROM FranquiaPublico);

In [ ]:
/* Filme que tem o elenco mais premiado com Oscars */

DROP TABLE IF EXISTS ElencoPremiado;

CREATE VIEW ElencoPremiado AS
    SELECT F.titulo, SUM(P.num_oscars) total_oscars
        FROM Pessoa P, Filme F, PessoaFilme PF
        WHERE P.id_TMDB = PF.id_pessoa
              AND F.id_TMDB = PF.id_filme
        GROUP BY F.titulo;

SELECT titulo
    FROM ElencoPremiado
    WHERE total_oscars = (SELECT MAX(total_oscars)
                              FROM ElencoPremiado);

In [ ]:
/* Diretores que produziram os melhores filmes, 
   de acordo com a crítica, do gênero drama */

DROP TABLE IF EXISTS DiretorFilmeDrama;

CREATE VIEW DiretorFilmeDrama AS
    SELECT P.nome, SUM(F.nota_media) nota_total, COUNT(P.nome) qtd_filme
        FROM Pessoa P, PessoaFilme PF, FilmeReceitaNota F, GeneroFilme G
        WHERE P.id_TMDB = PF.id_pessoa
              AND (PF.ator = False AND PF.diretor = True AND PF.roteirista = False)
              AND (PF.id_filme = F.id_filme AND G.id_filme)
              AND G.nome_genero = 'Drama'
        GROUP BY P.nome;

SELECT nome, (nota_total / qtd_filme) nota_media
    FROM DiretorFilmeDrama
    ORDER BY nota_media DESC LIMIT 10;

In [ ]:
/* Roteiristas que produziram os melhores filmes, 
   de acordo com a crítica, do gênero drama */

DROP TABLE IF EXISTS RoteiristaFilmeDrama;

CREATE VIEW RoteiristaFilmeDrama AS
    SELECT P.nome, SUM(F.nota_media) nota_total, COUNT(P.nome) qtd_filme
        FROM Pessoa P, PessoaFilme PF, FilmeReceitaNota F, GeneroFilme G
        WHERE P.id_TMDB = PF.id_pessoa
              AND (PF.ator = False AND PF.diretor = False AND PF.roteirista = True)
              AND (PF.id_filme = F.id_filme AND G.id_filme)
              AND G.nome_genero = 'Drama'
        GROUP BY P.nome;

SELECT nome, (nota_total / qtd_filme) nota_media
    FROM RoteiristaFilmeDrama
    ORDER BY nota_media DESC LIMIT 10;

In [ ]:
-- Receita e nota médias de cada gênero na década de 2000

DROP TABLE IF EXISTS GeneroNotaMedia;

CREATE VIEW GeneroNotaMedia AS
    SELECT G.nome_genero, SUM(F.receita) receita_total, SUM(F.nota_media) nota_total, COUNT(G.id_filme) qtd_filme
        FROM FilmeReceitaNota F, GeneroFilme G
        WHERE F.id_filme = G.id_filme
          AND (F.ano >= 2000 AND F.ano < 2010)
        GROUP BY G.nome_genero;

SELECT nome_genero, (receita_total / qtd_filme) receita_media, (nota_total / qtd_filme) nota_media
    FROM GeneroNotaMedia
    ORDER BY receita_media DESC;